## Imports

In [1]:
import numpy as np
from matplotlib import pyplot, cm
from mpl_toolkits.mplot3d import Axes3D

import time
from numba import njit
from pyccel.epyccel import epyccel
from pyccel.decorators import types

## Pure python function

In [2]:
def solve_2d_burger_pure(u, un, v, vn, nt, dt, dx, dy, nu):
    
    
    ###Assign initial conditions
    ##set hat function I.C. : u(.5<=x<=1 && .5<=y<=1 ) is 2
    u[int(.5 / dy):int(1 / dy + 1),int(.5 / dx):int(1 / dx + 1)] = 2 
    ##set hat function I.C. : u(.5<=x<=1 && .5<=y<=1 ) is 2
    v[int(.5 / dy):int(1 / dy + 1),int(.5 / dx):int(1 / dx + 1)] = 2
    row, col = u.shape
    
    #fill the update of u and v
    for n in range(nt+1):
        for i in range(row):
            for j in range(col):
                un[i,j] = u[i,j]
                vn[i,j] = v[i,j]
        
        u[1:-1,1:-1] = un[1:-1,1:-1] - \
        dt/dx * un[1:-1,1:-1] * (un[1:-1,1:-1]-un[0:-2,1:-1])-\
        dt/dy * vn[1:-1,1:-1] * (un[1:-1,1:-1] - un[1:-1,0:-2])+\
        nu * dt/dx**2 *(un[2:,1:-1] -2*un[1:-1,1:-1]+un[0:-2,1:-1])+\
        nu * dt/dy**2 * (un[1:-1,2:]-2*un[1:-1,1:-1] + un[1:-1,0:-2])
        
        v[1:-1,1:-1] = vn[1:-1,1:-1] - \
        dt/dx * un[1:-1,1:-1] * (vn[1:-1,1:-1]-vn[0:-2,1:-1])-\
        dt/dy * vn[1:-1,1:-1] * (vn[1:-1,1:-1] - vn[1:-1,0:-2])+\
        nu * dt/dx**2 *(vn[2:,1:-1] -2*vn[1:-1,1:-1]+vn[0:-2,1:-1])+\
        nu * dt/dy**2 * (vn[1:-1,2:]-2*vn[1:-1,1:-1] + vn[1:-1,0:-2])
        
        u[0,:]=1
        u[-1,:]=1
        u[:,0]=1
        u[:,-1]=1
        
        v[0,:]=1
        v[-1,:]=1
        v[:,0]=1
        v[:,-1]=1
        
    return 0

## Numba accelerator

In [3]:
@njit(fastmath=True)
def solve_2d_burger_numba(u, un, v, vn, nt, dt, dx, dy, nu):
    
    
    ###Assign initial conditions
    ##set hat function I.C. : u(.5<=x<=1 && .5<=y<=1 ) is 2
    u[int(.5 / dy):int(1 / dy + 1),int(.5 / dx):int(1 / dx + 1)] = 2 
    ##set hat function I.C. : u(.5<=x<=1 && .5<=y<=1 ) is 2
    v[int(.5 / dy):int(1 / dy + 1),int(.5 / dx):int(1 / dx + 1)] = 2
    row, col = u.shape
    
    
    for n in range(nt+1):
        for i in range(row):
            for j in range(col):
                un[i,j] = u[i,j]
                vn[i,j] = v[i,j]
        
        u[1:-1,1:-1] = un[1:-1,1:-1] - \
        dt/dx * un[1:-1,1:-1] * (un[1:-1,1:-1]-un[0:-2,1:-1])-\
        dt/dy * vn[1:-1,1:-1] * (un[1:-1,1:-1] - un[1:-1,0:-2])+\
        nu * dt/dx**2 *(un[2:,1:-1] -2*un[1:-1,1:-1]+un[0:-2,1:-1])+\
        nu * dt/dy**2 * (un[1:-1,2:]-2*un[1:-1,1:-1] + un[1:-1,0:-2])
        
        v[1:-1,1:-1] = vn[1:-1,1:-1] - \
        dt/dx * un[1:-1,1:-1] * (vn[1:-1,1:-1]-vn[0:-2,1:-1])-\
        dt/dy * vn[1:-1,1:-1] * (vn[1:-1,1:-1] - vn[1:-1,0:-2])+\
        nu * dt/dx**2 *(vn[2:,1:-1] -2*vn[1:-1,1:-1]+vn[0:-2,1:-1])+\
        nu * dt/dy**2 * (vn[1:-1,2:]-2*vn[1:-1,1:-1] + vn[1:-1,0:-2])
        
        u[0,:]=1
        u[-1,:]=1
        u[:,0]=1
        u[:,-1]=1
        
        v[0,:]=1
        v[-1,:]=1
        v[:,0]=1
        v[:,-1]=1
        
    return 0

## Pyccel accelerator

In [4]:

def solve_2d_burger_pyccel(u:'float[:,:]', un:'float[:,:]', v:'float[:,:]', vn:'float[:,:]', nt:'int', dt:'float', dx:'float', dy:'float', nu:'float'):    
    
    ###Assign initial conditions
    ##set hat function I.C. : u(.5<=x<=1 && .5<=y<=1 ) is 2
    u[int(.5 / dy):int(1 / dy + 1),int(.5 / dx):int(1 / dx + 1)] = 2 
    ##set hat function I.C. : u(.5<=x<=1 && .5<=y<=1 ) is 2
    v[int(.5 / dy):int(1 / dy + 1),int(.5 / dx):int(1 / dx + 1)] = 2
    row, col = u.shape
    
    #fill the update of u and v
    for n in range(nt+1):
        
        
        u[1:-1,1:-1] = un[1:-1,1:-1] - \
        dt/dx * un[1:-1,1:-1] * (un[1:-1,1:-1]-un[0:-2,1:-1])-\
        dt/dy * vn[1:-1,1:-1] * (un[1:-1,1:-1] - un[1:-1,0:-2])+\
        nu * dt/dx**2 *(un[2:,1:-1] -2*un[1:-1,1:-1]+un[0:-2,1:-1])+\
        nu * dt/dy**2 * (un[1:-1,2:]-2*un[1:-1,1:-1] + un[1:-1,0:-2])
        
        v[1:-1,1:-1] = vn[1:-1,1:-1] - \
        dt/dx * un[1:-1,1:-1] * (vn[1:-1,1:-1]-vn[0:-2,1:-1])-\
        dt/dy * vn[1:-1,1:-1] * (vn[1:-1,1:-1] - vn[1:-1,0:-2])+\
        nu * dt/dx**2 *(vn[2:,1:-1] -2*vn[1:-1,1:-1]+vn[0:-2,1:-1])+\
        nu * dt/dy**2 * (vn[1:-1,2:]-2*vn[1:-1,1:-1] + vn[1:-1,0:-2])
        
        u[0,:]=1
        u[-1,:]=1
        u[:,0]=1
        u[:,-1]=1
        
        v[0,:]=1
        v[-1,:]=1
        v[:,0]=1
        v[:,-1]=1
        
    return 0

## Inputs

In [5]:
###variable declarations
nx = 31
ny = 31
nt = 120
dx = 2 / (nx - 1)
dy = 2 / (ny - 1)
sigma = .0009
nu = 0.01
dt = sigma * dx * dy / nu


x = np.linspace(0, 2, nx)
y = np.linspace(0, 2, ny)

u = np.ones((ny, nx))  # create a 1xn vector of 1's
v = np.ones((ny, nx))
un = np.ones((ny, nx)) 
vn = np.ones((ny, nx))

solve_2d_burger_pure(u, un, v, vn, nt, dt, dx, dy, nu)

X, Y = np.meshgrid(x, y)
fig = pyplot.figure(figsize=(7, 5), dpi=100)
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, u, rstride=1, cstride=1, cmap=cm.viridis, linewidth=0, antialiased=True)
ax.set_zlim(1, 2.5)
ax.set_xlabel('$x$')
ax.set_ylabel('$y$');

/tmp/ipykernel_17598/326552352.py:24: MatplotlibDeprecationWarning: Calling gca() with keyword arguments was deprecated in Matplotlib 3.4. Starting two minor releases later, gca() will take no keyword arguments. The gca() function should only be used to get the current axes, or if no axes exist, create new axes with default keyword arguments. To create a new axes with non-default arguments, use plt.axes() or plt.subplot().
  ax = fig.gca(projection='3d')


## Call Funtions

### Pure function

In [6]:
%timeit solve_2d_burger_pure(u, un, v, vn, nt, dt, dx, dy, nu)

53.6 ms ± 833 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Function with numba accelerator

In [7]:
%timeit solve_2d_burger_numba(u, un, v, vn, nt, dt, dx, dy, nu)

3.4 ms ± 192 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Function with pyccel accelerator

In [8]:
solve_2d_burger_f90 = epyccel(solve_2d_burger_pyccel)

In [9]:
%timeit solve_2d_burger_f90(u, un, v, vn, nt, dt, dx, dy, nu)

365 µs ± 20.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
